In [ ]:
# Задача
# Спрогнозировать цену закрытия последней свечи торгового дня

In [280]:
import pandas as pd

In [281]:
data = pd.read_csv('SBERP.csv')

In [282]:
print(data[:10])

       DATE    TIME    OPEN    HIGH     LOW   CLOSE      VOL
0  20170301  110000  119.00  121.75  117.91  121.66  1102900
1  20170301  120000  121.66  122.71  121.29  122.34   986100
2  20170301  130000  122.35  123.21  122.22  123.01   329900
3  20170301  140000  123.03  123.14  122.15  122.35   280000
4  20170301  150000  122.35  123.19  122.10  123.13   266800
5  20170301  160000  123.13  123.49  122.87  122.92   378800
6  20170301  170000  122.88  123.29  122.35  123.27   374900
7  20170301  180000  123.27  123.37  122.62  123.26   367100
8  20170301  190000  123.21  123.50  122.92  122.92   673000
9  20170302  110000  123.28  125.09  123.28  125.00   962600


In [283]:
column_names = ["DATE",
               "TIME1", "OPEN1", "HIGH1", "LOW1", "CLOSE1", "VOL1",
               "TIME2", "OPEN2", "HIGH2", "LOW2", "CLOSE2", "VOL2",
               "TIME3", "OPEN3", "HIGH3", "LOW3", "CLOSE3", "VOL3",
               "TIME4", "OPEN4", "HIGH4", "LOW4", "CLOSE4", "VOL4",
               "TIME5", "OPEN5", "HIGH5", "LOW5", "CLOSE5", "VOL5",
               "TIME6", "OPEN6", "HIGH6", "LOW6", "CLOSE6", "VOL6",
               "TIME7", "OPEN7", "HIGH7", "LOW7", "CLOSE7", "VOL7",
               "TIME8", "OPEN8", "HIGH8", "LOW8", "CLOSE8", "VOL8",
               "TIME9", "OPEN9", "HIGH9", "LOW9", "CLOSE9", "VOL9"
              ]

def build_row_from_grouped_df(group_row):
    date = group_row[0]
    groups = group_row[1].values.tolist()
    
    tp = list()
    tp.append(date)
    
    for gr in groups:
        tp.append(gr[1])
        tp.append(gr[2])
        tp.append(gr[3])
        tp.append(gr[4])
        tp.append(gr[5])
        tp.append(gr[6])
        
    return tp


def group_and_reorganize(df, column_name):
    df_grouped = df.groupby(column_name)
    
    result = []
    for gr in df_grouped:
        result.append(build_row_from_grouped_df(gr))
        
    return pd.DataFrame(result, columns=column_names)

In [284]:
data = group_and_reorganize(data, ["DATE"])

In [286]:
train_data = data[:250].dropna()
test_data = data[250:].dropna()

In [287]:
train_features = train_data.drop(["DATE", "CLOSE9", "LOW9", "HIGH9", "VOL9","TIME1", "TIME2", "TIME3", "TIME4", 
                                  "TIME5", "TIME6", "TIME7", "TIME8", "TIME9"], axis=1)
train_target = train_data["CLOSE9"]

In [288]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)

knr = KNeighborsRegressor(n_neighbors=5, weights='distance')
knr.fit(train_features_scaled, train_target)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='distance')

In [289]:
test_features = test_data.drop(["DATE", "CLOSE9", "LOW9", "HIGH9", "VOL9","TIME1", "TIME2", "TIME3", "TIME4", 
                                  "TIME5", "TIME6", "TIME7", "TIME8", "TIME9"], axis=1)
test_target = test_data["CLOSE9"]
test_features_scaled = scaler.transform(test_features.dropna())

In [292]:
predictions = knr.predict(test_features_scaled)

from sklearn.metrics import r2_score
score = r2_score(test_target, predictions)
print(score)

0.8907803853544344


In [293]:
print(test_features[:1])

     OPEN1   HIGH1    LOW1  CLOSE1      VOL1   OPEN2  HIGH2    LOW2  CLOSE2  \
250  229.8  230.05  228.14   228.5  720400.0  228.52  229.5  228.25  228.45   

         VOL2  ...     HIGH7    LOW7  CLOSE7      VOL7  OPEN8   HIGH8    LOW8  \
250  254600.0  ...    226.72  225.16   225.4  248200.0  225.3  226.25  223.63   

     CLOSE8      VOL8  OPEN9  
250  225.96  693100.0  225.9  

[1 rows x 41 columns]


In [304]:
t1 = [[178.60, 179.50, 178.0400000,178.2800000,744400, 
      178.2900000,178.5300000,177.2300000,177.2500000,506000,
      177.2500000,178.3300000,177.0000000,178.2600000,559100,
      178.2600000,180.3000000,178.2600000,179.8000000,924000,
      179.8800000,180.6600000,179.3000000,180.4400000,558800,
      180.4400000,181.7700000,179.9200000,181.7300000,1542400,
      181.7700000,182.8400000,181.5700000,182.4000000,1109800,
      182.3700000,183.0500000,182.0000000,182.8800000,1168000,
      182.9600000]]

t1_scaled = scaler.transform(t1)
print(t1_scaled)

p1 = knr.predict(t1_scaled)
print(p1)


[[ 0.72678516  0.71806094  0.73646025  0.70860971  0.03624302  0.70901172
   0.69334253  0.69917141  0.67749111 -0.02165433  0.6775125   0.69053529
   0.68840856  0.70876397  0.39915207  0.70889042  0.75370392  0.72724239
   0.75624876  1.98008424  0.75878708  0.76782208  0.75867212  0.77712764
   0.8169817   0.77729984  0.80210667  0.77813982  0.8180726   3.48953852
   0.8193248   0.83446035  0.82767274  0.83526241  1.94459713  0.83451108
   0.8321506   0.84077066  0.84523671  1.21293853  0.84759989]]
[188.23109623]


In [305]:
t2 = [[ 213.6900000,215.1600000,213.5900000,214.7200000,701500,
       214.8300000,215.5000000,214.4000000,215.2800000,273700,
       215.3900000,218.0000000,215.2000000,215.5000000,467800,
       215.3600000,216.1700000,214.8500000,215.2000000,190800,
       215.1700000,215.6200000,214.0200000,214.9500000,233200,
       214.9500000,214.9600000,214.3300000,214.6200000,72000,
       214.7600000,216.2300000,214.6700000,215.9800000,132400,
       215.9800000,215.9900000,214.4200000,214.9700000,491400,
       214.9700000]]

t2_scaled = scaler.transform(t2)
print(t2_scaled)

p2 = knr.predict(t2_scaled)
print(p2)

[[ 1.80623788  1.80571226  1.83174505  1.82402148 -0.03618764  1.82766475
   1.81993745  1.84260562  1.84218085 -0.59002419  1.84565229  1.90049709
   1.86237215  1.84912037  0.12660264  1.84502305  1.84784889  1.85272749
   1.84098334 -0.5432262   1.84010201  1.8368425   1.82747761  1.83588381
  -0.35122162  1.83628393  1.81754852  1.83664553  1.8270306  -0.86975198
   1.83142851  1.85673215  1.84441388  1.86499227 -0.87950874  1.86536161
   1.83634604  1.83806846  1.82689484 -0.0730517   1.82680975]]
[209.88370956]
